In [1]:
import os
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

# Define paths
images_path = 'GTA5/GTA5/images'
labels_path = 'GTA5/GTA5/labels'

# Initialize lists to hold data
data = []

# Load images and corresponding masks
for image_filename in os.listdir(images_path):
    if image_filename.endswith('.png'):
        image_path = os.path.join(images_path, image_filename)
        mask_path = os.path.join(labels_path, image_filename)
        
        # Check if corresponding mask file exists
        if os.path.exists(mask_path):
            # Open image and mask to ensure they can be loaded (optional, for validation)
            try:
                image = Image.open(image_path)
                mask = Image.open(mask_path)
                
                # Add data to list
                data.append({
                    'image_path': image_path,
                    'mask_path': mask_path
                })
            except Exception as e:
                print(f"Error loading {image_path} or {mask_path}: {e}")

# Create a DataFrame from the data list
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv('gta5_segmentation_dataset.csv', index=False)

print("Semantic segmentation dataset created and saved as 'gta5_segmentation_dataset.csv'")

Semantic segmentation dataset created and saved as 'gta5_segmentation_dataset.csv'


In [2]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from pytorchdl_gta5.labels import GTA5Labels_TaskCV2017
import albumentations as A
from albumentations.pytorch import ToTensorV2

if __name__ != '__main__':
    raise Exception("This script should not be imported; it should be run directly.")

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define the custom dataset class
class GTASegmentationDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform
        self.label_mapping = self._create_label_mapping()
        self.color_jitter = T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
        self.gaussian_blur = T.GaussianBlur(kernel_size=(3, 7), sigma=(0.1, 5))

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
    
        img_name = self.data_frame.iloc[idx, 0]
        mask_name = self.data_frame.iloc[idx, 1]
    
        image = Image.open(img_name).convert('RGB')
        mask = Image.open(mask_name).convert('RGB')
        
        image = self.color_jitter(image)

        image = self.gaussian_blur(image)
        
        if self.transform:
            augmented = self.transform(image=np.array(image), mask=np.array(mask))
            image, mask = augmented['image'], augmented['mask']
            
        mask = self._map_mask(np.array(mask))
    
        # Convert mask to tensor without normalization
        mask = torch.from_numpy(mask).long()  # Ensure the mask is of type long for cross-entropy loss
    
        return image, mask

    def _create_label_mapping(self):
        label_mapping = {label.color: label.ID for label in GTA5Labels_TaskCV2017.list_}
        label_mapping[(0, 0, 0)] = 255  # Ensure unmapped colors go to 'unlabeled'
        return label_mapping
        
    def _map_mask(self, mask):
        new_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.uint8)
        for color, label_id in self.label_mapping.items():
            color_mask = np.all(mask == color, axis=-1)
            new_mask[color_mask] = label_id  # Use label_id instead of color
        return new_mask

# Define paths
csv_file = 'gta5_segmentation_dataset.csv'

# Define image transformations
transform = A.Compose([
    A.Resize(720, 1280),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    A.RandomRotate90(),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomResizedCrop(height=720, width=1280, scale=(0.8, 1.0)),
    ToTensorV2()
])

# Create the dataset and dataloader
train_dataset = GTASegmentationDataset(csv_file=csv_file, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
"""
import os
import pandas as pd
from PIL import Image
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from pytorchdl_gta5.labels import GTA5Labels_TaskCV2017
import albumentations as A
from albumentations.pytorch import ToTensorV2

if __name__ != '__main__':
    raise Exception("This script should not be imported; it should be run directly.")

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define the custom dataset class
class GTASegmentationDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform
        self.label_mapping = self._create_label_mapping()

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
    
        img_name = self.data_frame.iloc[idx, 0]
        mask_name = self.data_frame.iloc[idx, 1]
    
        image = Image.open(img_name).convert('RGB')
        mask = Image.open(mask_name).convert('RGB')
        
        if self.transform:
            augmented = self.transform(image=np.array(image), mask=np.array(mask))
            image, mask = augmented['image'], augmented['mask']
            
        mask = self._map_mask(np.array(mask))
    
        # Convert mask to tensor without normalization
        mask = torch.from_numpy(mask).long()  # Ensure the mask is of type long for cross-entropy loss
    
        return image, mask

    def _create_label_mapping(self):
        label_mapping = {label.color: label.ID for label in GTA5Labels_TaskCV2017.list_}
        label_mapping[(0, 0, 0)] = 255  # Ensure unmapped colors go to 'unlabeled'
        return label_mapping
        
    def _map_mask(self, mask):
        new_mask = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.uint8)
        for color, label_id in self.label_mapping.items():
            color_mask = np.all(mask == color, axis=-1)
            new_mask[color_mask] = label_id  # Use label_id instead of color
        return new_mask

# Define paths
csv_file = 'gta5_segmentation_dataset.csv'

# Define image transformations
transform = A.Compose([
    A.Resize(512, 1024),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    A.RandomRotate90(),
    A.RandomResizedCrop(height=512, width=1024, scale=(0.8, 1.0)),
    ToTensorV2()
])

# Create the dataset and dataloader
train_dataset = GTASegmentationDataset(csv_file=csv_file, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
"""

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.8 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations


'\nimport os\nimport pandas as pd\nfrom PIL import Image\nimport numpy as np\nimport torch\nfrom torch.utils.data import Dataset, DataLoader\nfrom torchvision import transforms\nfrom pytorchdl_gta5.labels import GTA5Labels_TaskCV2017\nimport albumentations as A\nfrom albumentations.pytorch import ToTensorV2\n\nif __name__ != \'__main__\':\n    raise Exception("This script should not be imported; it should be run directly.")\n\n# Setup device agnostic code\ndevice = "cuda" if torch.cuda.is_available() else "cpu"\n\n# Define the custom dataset class\nclass GTASegmentationDataset(Dataset):\n    def __init__(self, csv_file, transform=None):\n        self.data_frame = pd.read_csv(csv_file)\n        self.transform = transform\n        self.label_mapping = self._create_label_mapping()\n\n    def __len__(self):\n        return len(self.data_frame)\n\n    def __getitem__(self, idx):\n        if torch.is_tensor(idx):\n            idx = idx.tolist()\n    \n        img_name = self.data_frame.iloc[

In [3]:
import numpy as np
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as T
from torch.utils.data import DataLoader
import os


class Cityscapes(Dataset):
    def __init__(self, root_dir, split, transforms=None, label_type='gtFine_labelTrainIds'):
        self.root_dir = root_dir
        self.split = split
        self.transforms = transforms
        self.label_type = label_type

        self.images_dir = f"{root_dir}/images/{split}"
        self.labels_dir = f"{root_dir}/gtFine/{split}"

        self.image_paths = []
        self.label_paths = []

        # Manually iterate over directories
        cities = [city for city in os.listdir(self.images_dir) if os.path.isdir(f"{self.images_dir}/{city}")]
        for city in cities:
            img_dir_city = f"{self.images_dir}/{city}"
            lbl_dir_city = f"{self.labels_dir}/{city}"

            if not os.path.isdir(img_dir_city) or not os.path.isdir(lbl_dir_city):
                continue

            for img_file in os.listdir(img_dir_city):
                if img_file.endswith('_leftImg8bit.png'):
                    img_path = f"{img_dir_city}/{img_file}"
                    lbl_file = img_file.replace('_leftImg8bit.png', f'_{self.label_type}.png')
                    lbl_path = f"{lbl_dir_city}/{lbl_file}"

                    if os.path.isfile(img_path) and os.path.isfile(lbl_path):
                        self.image_paths.append(img_path)
                        self.label_paths.append(lbl_path)
                    else:
                        print(f"Warning: Image or label file not found for {img_file}")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        lbl_path = self.label_paths[idx]

        image = Image.open(img_path).convert('RGB')
        label = Image.open(lbl_path)

        image = np.array(image)
        label = np.array(label)
        
        if self.transforms:
            augmented = self.transforms(image=image, mask=label)
            image, label = augmented['image'], augmented['mask']
        
        return image, label


# Example usage
image_transforms = A.Compose([
    A.Resize(512, 1024),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_dataset = Cityscapes(root_dir='Cityscapes/Cityspaces', split='val', transforms=image_transforms)

val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=False)

print(f"Val_Dataset size: {len(val_dataset)}")

Val_Dataset size: 500


In [4]:
from build_bisenet import BiSeNet
from torch import nn

# Set CUDA_LAUNCH_BLOCKING environment variable
os.environ['CUDA_LAUNCH_BLOCKING']="1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

context_path = 'resnet18'

# Initialize the model
model = BiSeNet(num_classes=19, context_path=context_path).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Set the manual seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [5]:
def train(model, optimizer_train, dataloader, loss_fn_train):
    model.train()  # Set the model to training mode
    train_loss = 0.0
    total = 0

    for idx, (inputs_train, targets_train) in enumerate(dataloader):
        inputs_train = inputs_train.to(device)
        targets_train = targets_train.to(device, dtype=torch.long)  # Move data to the appropriate device

        optimizer_train.zero_grad()  # Zero out gradients from the previous iteration
        outputs_train, cx1_sup, cx2_sup = model(inputs_train)  # Forward pass
        # print(outputs_train.shape, targets_train.shape)
        loss = loss_fn_train(outputs_train, targets_train)  # Calculate the loss
        
        aux_loss1 = loss_fn_train(cx1_sup, targets_train)
        aux_loss2 = loss_fn_train(cx2_sup, targets_train)

        loss = loss + aux_loss1 + aux_loss2
        
        loss.backward()  # Backward pass
        optimizer_train.step()  # Update the weights

        train_loss += loss.item() * inputs_train.size(0)  # Accumulate the total loss
        _, predicted_train = outputs_train.max(1)
        total += targets_train.size(0)

    # Calculate average loss for the epoch
    avg_loss = train_loss / total
    
    return avg_loss


def compute_iou(pred, target, num_classes):
    ious = []
    pred = pred.view(-1)
    target = target.view(-1)
    
    for cls in range(num_classes):
        pred_inds = (pred == cls)
        target_inds = (target == cls)
        intersection = (pred_inds[target_inds]).sum().item()
        union = pred_inds.sum().item() + target_inds.sum().item() - intersection
        if union == 0:
            ious.append(float('nan'))  # If there is no union, set IoU to NaN
        else:
            ious.append(intersection / union)
    
    return np.array(ious)

def eval(model, dataloader, loss_fn, device, num_classes=19):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0.0
    total = 0
    all_ious = []  # List to store IoUs for each batch
    
    with torch.no_grad():  # Disable gradient calculation during inference
        for inputs_test, targets_test in dataloader:
            inputs_test, targets_test = inputs_test.to(device), targets_test.to(device, dtype=torch.long)

            outputs_test = model(inputs_test)  # Forward pass
            loss = loss_fn(outputs_test, targets_test)  # Calculate the loss

            test_loss += loss.item() * inputs_test.size(0)  # Accumulate the total loss
            _, predicted_test = outputs_test.max(1)
            total += targets_test.size(0)

            # Compute IoU for this batch
            batch_ious = compute_iou(predicted_test, targets_test, num_classes)
            all_ious.append(batch_ious)
    
    # Calculate average loss
    avg_loss = test_loss / total
    
    # Calculate mean IoU
    all_ious = np.array(all_ious)
    mean_iou = np.nanmean(all_ious, axis=0)  # Mean IoU for each class
    miou = np.nanmean(mean_iou)  # Mean IoU across all classes
    
    return avg_loss, miou

In [ ]:
from timeit import default_timer as timer
start_time = timer()

# Setup training and save the results
for _ in range(50):
    train(model, optimizer, train_dataloader, loss_fn)
    avg_loss, miou = eval(model, val_dataloader, loss_fn, device=device)
    print(f"Loss: {avg_loss}, mIoU: {miou*100:.2f}%")

# End the timer and print out how long it took
end_time = timer()
print(f"[INFO] Total training time: {end_time - start_time:.3f} seconds")

Loss: 1.3065281097888946, mIoU: 11.56%
Loss: 1.183289046049118, mIoU: 13.44%
Loss: 1.749862690448761, mIoU: 10.87%
Loss: 1.3642717125415802, mIoU: 12.32%
Loss: 1.002148030757904, mIoU: 14.78%
Loss: 0.9140036593675613, mIoU: 15.88%
Loss: 1.1960161559581757, mIoU: 14.09%
Loss: 1.3787701768875122, mIoU: 13.96%
Loss: 1.4470102529525757, mIoU: 13.73%
Loss: 1.3000136684179306, mIoU: 14.78%
Loss: 1.5022656930685043, mIoU: 13.59%
Loss: 1.2361995618343353, mIoU: 14.77%
Loss: 1.0245310797691345, mIoU: 15.83%
Loss: 0.9273065032958985, mIoU: 17.04%
Loss: 1.2353467073440552, mIoU: 14.16%
Loss: 1.3709070225954056, mIoU: 15.59%
Loss: 1.0029244220256806, mIoU: 16.02%
Loss: 1.2437726366519928, mIoU: 14.36%
Loss: 1.307647983789444, mIoU: 13.18%
Loss: 1.0403612014055252, mIoU: 16.15%
Loss: 1.048096360206604, mIoU: 16.35%
Loss: 0.832539591908455, mIoU: 18.70%
Loss: 1.220076737523079, mIoU: 15.20%
Loss: 0.8883176387548447, mIoU: 18.59%
Loss: 1.1352757929563522, mIoU: 16.27%
Loss: 1.6325649864673615, mIoU: 